### Test1

In [1]:
import ads
import pandas as pd

from FlagEmbedding import FlagReranker

# to save in the Model Catalog
from ads.model.generic_model import GenericModel
from ads.model.model_metadata import MetadataCustomCategory

In [2]:
ads.set_auth(auth="resource_principal")

In [3]:
# read file for input schema definition
dati = pd.read_csv("dati.csv")

X = dati[["x1", "x2"]]
y = dati["score"]

In [4]:
#
# This custom class wrap the reranker model
#
class Reranker:
    def __init__(self, model_id):
        self.model_id = model_id
        self.reranker = FlagReranker(self.model_id, use_fp16=True)

    def predict(self, x):
        # x is expected as a list of list of str
        # [["x1", "x2"]] -> y = [score12]
        scores = self.reranker.compute_score(x)

        return scores

#### Test the model locally

In [5]:
model = Reranker(model_id="BAAI/bge-reranker-large")

  warn("The installed version of bitsandbytes was compiled without GPU support. "

/home/datascience/conda/reranker_env_v2_0/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpjyt0p9tu
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpjyt0p9tu/_remote_module_non_scriptable.py
[2023-12-31 17:45:05,712] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [6]:
model.predict([["Input1", "Input2"]])

-1.9253941774368286

#### Prepare to save the model to the Model Catalog

In [7]:
reranker_model = GenericModel(estimator=model, artifact_dir="./reranker_dir")

In [8]:
reranker_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [9]:
# this is the path of the custom_conda_env that must be used for the deployment runtime (contains aditional libraries)
INF_ENV_PATH = "oci://conda_envs2024@frqap2zhtzbe/conda_environments/gpu/reranker_env_/2.0/reranker_env_v2_0"

# here we prepare and check artefacts for deployment
reranker_model.prepare(
    reload=False,
    inference_conda_env=INF_ENV_PATH,
    inference_python_version="3.9",
    model_file_name="reranker.pkl",
    force_overwrite=True,
    X_sample=X,
    y_sample=y,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


algorithm: null
artifact_dir:
  /home/datascience/reranker_dir:
  - - test_json_output.json
    - reranker.pkl
    - input_schema.json
    - score.py
    - runtime.yaml
    - output_schema.json
    - .model-ignore
    - .ipynb_checkpoints
    - .ipynb_checkpoints/output_schema-checkpoint.json
    - .ipynb_checkpoints/input_schema-checkpoint.json
    - .ipynb_checkpoints/test_json_output-checkpoint.json
framework: null
model_deployment_id: null
model_id: null

In [10]:
test1 = [
    ["what is panda?", "It is an animal living in China"],
    ["what is panda?", "The giant panda is a bear species endemic to China."],
]

reranker_model.verify(test1)

INFO:root:Start loading reranker.pkl from model directory /home/datascience/reranker_dir ...
INFO:root:Model is successfully loaded.
INFO:root:Predict with input data: [['what is panda?', 'It is an animal living in China'], ['what is panda?', 'The giant panda is a bear species endemic to China.']]...
INFO:root:Input data type is: ...


{'prediction': [1.1291378736495972, 4.953792095184326]}

In [11]:
reranker_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

#### save to the Model Catalog

In [12]:
# define some custom metadata
reranker_model.metadata_custom.add(
    key="reranker",
    value="FlagEmbedding large",
    category=MetadataCustomCategory.OTHER,
    description="reranker",
    replace=True,
)

In [13]:
model_id = reranker_model.save(
    display_name="reranker2-baai-large",
    bucket_uri="oci://reranker_models@frqap2zhtzbe/reranker/",
)

INFO:root:Start loading reranker.pkl from model directory /home/datascience/reranker_dir ...
INFO:root:Model is successfully loaded.
['test_json_output.json', 'reranker.pkl', 'input_schema.json', 'score.py', 'runtime.yaml', 'output_schema.json', '.model-ignore']


loop1:   0%|          | 0/7 [00:00<?, ?it/s]

/tmp/tmpyshz5wtv.zip has been successfully uploaded to oci://reranker_models@frqap2zhtzbe/reranker/ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaangencdyana5crgez6zl4pau2wwbqmpab2z4fn4wfj26nq6ikckkq.zip.


loop1:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
reranker_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

#### Create the Model Deployment

In [15]:
reranker_model.deploy(
    display_name="reranker2-baai-large",
    deployment_log_group_id="ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdya63i3qhao4bjx754lb3m2jpekev5oc55p5ebjvykbtgya",
    deployment_access_log_id="ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdyamg6gyfdjofod7hlnbhhjtgeaeyy3fkrmh3cyb4dxx7xa",
    deployment_predict_log_id="ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdyaddqi3rff7kdbxhxdpi2rx65dynuye36dayz7nivbwsca",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=2,
    deployment_memory_in_gbs=64,
)

loop1:   0%|          | 0/100 [00:00<?, ?it/s]


kind: deployment
spec:
  createdBy: ocid1.datasciencenotebooksession.oc1.eu-frankfurt-1.amaaaaaangencdyap5ukzx25372kkwgywuz76vtuiqawaqhrqg5o5s5medla
  definedTags:
    default-tags:
      CreatedBy: ocid1.datasciencenotebooksession.oc1.eu-frankfurt-1.amaaaaaangencdyap5ukzx25372kkwgywuz76vtuiqawaqhrqg5o5s5medla
  displayName: reranker2-baai-large
  id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaokm6zawt3akgu3lr7u4hm4o4zrr64emfr3vi2qmzw2xa
  infrastructure:
    kind: infrastructure
    spec:
      accessLog:
        logGroupId: ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdya63i3qhao4bjx754lb3m2jpekev5oc55p5ebjvykbtgya
        logId: ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdyamg6gyfdjofod7hlnbhhjtgeaeyy3fkrmh3cyb4dxx7xa
      bandwidthMbps: 10
      compartmentId: ocid1.compartment.oc1..aaaaaaaag2cpni5qj6li5ny6ehuahhepbpveopobooayqfeudqygdtfe6h3a
      deploymentType: SINGLE_MODEL
      policyType: FIXED_SIZE
      predictLog:
        logGroupId: ocid1.loggr

In [16]:
model_deployment_url = reranker_model.model_deployment.url

model_deployment_url

'https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaokm6zawt3akgu3lr7u4hm4o4zrr64emfr3vi2qmzw2xa'

In [17]:
reranker_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

#### Test the Model Deployment

In [21]:
%%time

test2 = [
    ["what is panda?", "The giant panda is a bear species endemic to China."],
    ["what is panda?", "It is an animal living in China"],
]

results = reranker_model.predict(test2)

results["prediction"]

CPU times: user 361 ms, sys: 0 ns, total: 361 ms
Wall time: 968 ms


[4.953792095184326, 1.129137396812439]